#### Objective

The objective of this project is to calculate the raw material cost for all part codes contained in an Excel spreadsheet, whether the code refers to a part, welded set, or assembled set.

##### Motivation

This idea arises from the need to calculate non-conformity costs. Since the parts of internal projects follow a different workflow when entering the factory compared to projects handled by the outsourced engineering company, the former are not present in the SteelProjects parts database, the software used for programming and managing factory projects.

As a result, any non-conformity involving parts from internal projects requires manually researching and entering their weight, raw material, and dimensions into an Excel spreadsheet so that their cost can be correctly calculated in the non-conformity cost sheet. This problem is compounded when the non-conformities involve welded or assembled sets, where the correct approach would be to calculate the unit cost of each of its components and sum the costs. However, due to the large amount of time required to do this, the part's cost is calculated by simply multiplying the total weight of the set by an average value per kilogram from various products.

##### Methodology

To obtain the data for parts from internal projects, the project list generated by the engineering team will be used. This list contains almost all the necessary information for the calculation, such as whether the code refers to a single part, welded set, or assembled set, raw material, dimensions, weight, and the ERP code related to the raw material. To obtain the raw material cost from the ERP code, it will be necessary to search in another spreadsheet that extracts this information from the ERP.

##### Tools
For this project, the well-known "pandas" library, already explained in class, will be used, along with the "openpyxl" library, which was not taught in class. The need for the latter arises because the distinction between welded and assembled sets is made based on the color of the main row of the set. This is necessary because the openpyxl library works with properties of the Excel spreadsheet in addition to the data, allowing it to collect the color of rows, while pandas cannot.

##### Spreadsheet Logic

To determine which components are part of each set, the following logic is adopted by the engineering team:

- Blue row (theme = 8): Corresponds to the project, everything between two blue rows corresponds to the components of the project;
- Gray row (theme = 2): Corresponds to a welded set, everything between the gray row and the next colored row is part of the welded set;
- Green row (theme = 9): Corresponds to an assembled set; everything between the green row and the next colored row is part of the assembled set;
- There are no assembled sets within welded sets;
- There are no welded sets within other welded sets;
- If there is a welded set within an assembled set, the part codes that make up the welded set will appear as components of the assembled set, but the welded set code will not be listed as a component of the assembled set;
- The "code" and "component" columns do not have any distinction, they are just used to differentiate what is a set/project from what is a component of these. These columns might not be filled out as originally intended. However, what defines whether it is a welded set, assembled set, or project, and the parts that make them up, is the color of the row.

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
# Reading the Spreadsheet with openpyxl

workbook = load_workbook(r"C:\Users\Guilherme Contar\OneDrive - HTMS Group\Documentos\Trabalho python\Planilha\P003.0005.A001_E00.xlsm")
ws = workbook["Lista de Materiais"]

In [3]:
# Saving the Rows of the Spreadsheet in a List

lista_ws = []
for row in ws.iter_rows(values_only=True):
    lista_ws.append(row)

print(lista_ws)


[(None, 'Nº DOCUMENTO:\nP003.0005.A001_E00', None, 'LISTA DE MATERIAIS\nPOSTE SLS 9M V045', None, None, None, None, None, None, None, None, 'Nº DA REVISÃO:\nREV-2024-00', None, None), ('MULTIPLICADOR:\n', 1, None, 'SITE:', None, None, None, None, None, 'REVISADO:\n', None, None, 'APROVADO:\n', 'EVERTOM W.', 'DATA:'), ('CÓDIGO', 'COMPONENTE', 'DESCRIÇÃO', 'CÓDIGO SANKHYA', 'CARACTERÍSTICAS', 'MATERIAL', 'ESP.', 'QDT. UNIT.', 'QDT. TOTAL', 'PESO KG', 'PESO TOTAL', 'COMPRIMENTO', 'LARGURA', 'TRATAMENTO', 'PINTURA'), (None, 'P003.0005.M001_E00', 'CHUMBADOR', None, 'MONTAGEM FINAL', 'N/A', 'N/A', 1, 1, 39, 39, 583, 583, 'N/A', 'N/A'), ('AR.LS.P19.05', None, 'FIXAÇÕES', 538, 'ARRUELA LISA 3/4"', 'ASTM-A325', 'N/A', 10, 10, 0, 0, 0, 0, 'GALV. A FOGO', 'N/A'), ('PO.SX.P19.05-10', None, 'FIXAÇÕES', 1838, 'PORCA SEXT. 3/4"-10', 'ASTM-A325', 'N/A', 30, 30, 0, 0, 0, 0, 'GALV. A FOGO', 'N/A'), ('  P003.0005.S010_E00', None, 'CHUMBADOR', None, 'SOLDAGEM', 'N/A', 'N/A', 1, 1, 37.2, 37.2, 583, 583, 'G

In [ ]:
# Since the library cannot access the color of rows or columns, only cells, I use another for loop to go through only the cells in the first column, which returns the color of the row (if the entire row is painted the same color)

lista_cores = []
for cell in ws['A']:
    cor = cell.fill.fgColor.theme # Get the treated color, values with text are cells without colors
    print(cell.fill.fgColor)
    lista_cores.append(cor) 
    
print(lista_cores) # Cells that are not painted do not have .theme, that's why this "error" appears

lista_cores = lista_cores[3:] # Removing the first two rows and the header
lista_cores = [0 if type(i) != int else i for i in lista_cores] # Transforming the warning that appears for unpainted cells (the warning is not saved as a string, but as an openpyxl object)

print(lista_cores)


<openpyxl.styles.colors.Color object>
Parameters:
rgb='00000000', indexed=None, auto=None, theme=None, tint=0.0, type='rgb'
<openpyxl.styles.colors.Color object>
Parameters:
rgb='00000000', indexed=None, auto=None, theme=None, tint=0.0, type='rgb'
<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=1, tint=0.1499984740745262, type='theme'
<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=8, tint=0.5999938962981048, type='theme'
<openpyxl.styles.colors.Color object>
Parameters:
rgb='00000000', indexed=None, auto=None, theme=None, tint=0.0, type='rgb'
<openpyxl.styles.colors.Color object>
Parameters:
rgb='00000000', indexed=None, auto=None, theme=None, tint=0.0, type='rgb'
<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=2, tint=-0.249977111117893, type='theme'
<openpyxl.styles.colors.Color object>
Parameters:
rgb='00000000', indexed=None, auto=None, theme=None, tint=

In [5]:
# Tranforming the list into Dataframe

df = pd.DataFrame(lista_ws[2:])
df.columns = df.iloc[0]
df = df[1:]
df

,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA
1,None,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1,39,39,583,583,N/A,N/A
2,AR.LS.P19.05,None,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10,0,0,0,0,GALV. A FOGO,N/A
3,PO.SX.P19.05-10,None,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30,0,0,0,0,GALV. A FOGO,N/A
4,P003.0005.S010_E00,None,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1,37.2,37.2,583,583,GALV. A FRIO,N/A
5,None,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20,1.09,21.8,277.22,80,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0018.0001.M003_R00,None,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1,0,0,0,0,N/A,N/A
258,P003.0005.P223_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.06,0.06,207.5,35,N/A,N/A
259,P003.0005.P229_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.03,0.03,105,35,N/A,N/A
260,P003.0005.P533_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3,0.02,0.06,67.5,35,N/A,N/A


In [6]:
# Correcting the table

df["CÓDIGO"] = df["CÓDIGO"].str.strip() # Remove spaces at the beginning and end of the text
df["CÓDIGO"] = df["CÓDIGO"].str.replace(" ","",regex=False) # Removes spaces in the middle of the text (possible typing errors)
df["CÓDIGO"] = df["CÓDIGO"].fillna("") # Filling empty cells with empty strings so the "ÉPeça?" column conditional works

df["COMPONENTE"] = df["COMPONENTE"].str.strip()
df["COMPONENTE"] = df["COMPONENTE"].str.replace(" ","",regex=False)
df["COMPONENTE"] = df["COMPONENTE"].fillna("")

df["QDT. TOTAL"] = pd.to_numeric(df["QDT. TOTAL"], errors="coerce")
df["PESO TOTAL"] = pd.to_numeric(df["PESO TOTAL"], errors="coerce")

df


,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA
1,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1.0,39,39.00,583,583,N/A,N/A
2,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10.0,0,0.00,0,0,GALV. A FOGO,N/A
3,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30.0,0,0.00,0,0,GALV. A FOGO,N/A
4,P003.0005.S010_E00,,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1.0,37.2,37.20,583,583,GALV. A FRIO,N/A
5,,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20.0,1.09,21.80,277.22,80,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0018.0001.M003_R00,,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1.0,0,0.00,0,0,N/A,N/A
258,P003.0005.P223_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.06,0.06,207.5,35,N/A,N/A
259,P003.0005.P229_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.03,0.03,105,35,N/A,N/A
260,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3.0,0.02,0.06,67.5,35,N/A,N/A


In [7]:
# Pairing rows with the color list
df["Cor"] = lista_cores

df

,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA,Cor
1,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1.0,39,39.00,583,583,N/A,N/A,8
2,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10.0,0,0.00,0,0,GALV. A FOGO,N/A,0
3,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30.0,0,0.00,0,0,GALV. A FOGO,N/A,0
4,P003.0005.S010_E00,,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1.0,37.2,37.20,583,583,GALV. A FRIO,N/A,2
5,,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20.0,1.09,21.80,277.22,80,N/A,N/A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0018.0001.M003_R00,,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1.0,0,0.00,0,0,N/A,N/A,0
258,P003.0005.P223_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.06,0.06,207.5,35,N/A,N/A,0
259,P003.0005.P229_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.03,0.03,105,35,N/A,N/A,0
260,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3.0,0.02,0.06,67.5,35,N/A,N/A,0


In [8]:
# Defines what is a part and what is not

regex = r'^[A-Za-z0-9]{4}\.[A-Za-z0-9]{4}\.[A-Za-z0-9]{4}_[A-Za-z0-9]{3}$' # Sets the text format that will define a part code

df["ÉPeça?"] = df["CÓDIGO"].str.match(regex) | df["COMPONENTE"].str.match(regex)

df


,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA,Cor,ÉPeça?
1,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1.0,39,39.00,583,583,N/A,N/A,8,True
2,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10.0,0,0.00,0,0,GALV. A FOGO,N/A,0,False
3,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30.0,0,0.00,0,0,GALV. A FOGO,N/A,0,False
4,P003.0005.S010_E00,,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1.0,37.2,37.20,583,583,GALV. A FRIO,N/A,2,True
5,,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20.0,1.09,21.80,277.22,80,N/A,N/A,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0018.0001.M003_R00,,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1.0,0,0.00,0,0,N/A,N/A,0,True
258,P003.0005.P223_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.06,0.06,207.5,35,N/A,N/A,0,True
259,P003.0005.P229_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.03,0.03,105,35,N/A,N/A,0,True
260,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3.0,0.02,0.06,67.5,35,N/A,N/A,0,True


In [9]:
# Importing cost table
df_custos = pd.read_excel(r"C:\Users\Guilherme Contar\OneDrive - HTMS Group\Qualidade\SGQ\CONTROLES\RNC\Custos Sankhya\CustosSankhyaFinal.xlsx")

df_custos["Data Atualização"] = pd.to_datetime(df_custos["Data Atualização"])

# Selecting the most recent row for each "CODPROD"
df_custos_unicos = df_custos.loc[df_custos.groupby("CODPROD")["Data Atualização"].idxmax()]

# Merging based on CODPROD
df = df.merge(
    df_custos_unicos[["CODPROD", "Custo Médio com ICMS"]],
    how="left",
    left_on="CÓDIGO SANKHYA",
    right_on="CODPROD"
)

df["Custo Médio com ICMS"] = pd.to_numeric(df["Custo Médio com ICMS"],errors="coerce")
df["Custo Médio com ICMS"] = df["Custo Médio com ICMS"].fillna(0)

df


,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA,Cor,ÉPeça?,CODPROD,Custo Médio com ICMS
0,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1.0,39,39.00,583,583,N/A,N/A,8,True,NaN,0.00
1,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10.0,0,0.00,0,0,GALV. A FOGO,N/A,0,False,538.0,7.36
2,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30.0,0,0.00,0,0,GALV. A FOGO,N/A,0,False,1838.0,1.92
3,P003.0005.S010_E00,,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1.0,37.2,37.20,583,583,GALV. A FRIO,N/A,2,True,NaN,0.00
4,,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20.0,1.09,21.80,277.22,80,N/A,N/A,0,True,154.0,5.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,0018.0001.M003_R00,,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1.0,0,0.00,0,0,N/A,N/A,0,True,9031.0,26.90
257,P003.0005.P223_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.06,0.06,207.5,35,N/A,N/A,0,True,1033.0,7.57
258,P003.0005.P229_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1.0,0.03,0.03,105,35,N/A,N/A,0,True,1033.0,7.57
259,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3.0,0.02,0.06,67.5,35,N/A,N/A,0,True,1033.0,7.57


In [10]:
# Calculating the cost of each component; if it is a part, multiply the unit cost by the weight, otherwise by the quantity

df["Custo Total"] = np.where(
    df["ÉPeça?"] == False,  # Condition
    df["Custo Médio com ICMS"] * df["QDT. TOTAL"],
    df["Custo Médio com ICMS"] * df["PESO TOTAL"] 
)


In [11]:
# List with the values of the colored rows
lista_linhas_coloridas_valores = np.array([i for i in lista_cores if i != 0])

# List with the table index of the respective color
lista_linhas_coloridas_indices = np.array([i for i in range(len(lista_cores)) if lista_cores[i] != 0])

print("Sequence of colored rows:", lista_linhas_coloridas_valores)
print("Indices of colored rows:", lista_linhas_coloridas_indices)


Sequencia de linhas coloridas: [8 2 8 2 9 2 2 9 2 9 9 9 9 9 9 9 8 2 2 9 2 8 2 9 2 9 2 9]
Índices das linhas coloridas: [  0   3   7  16  47  53  58  61  66  69  79  86  93 107 119 129 141 151
 165 170 172 176 185 201 203 207 212 215]


In [12]:
# Calculating projects

cond = (lista_linhas_coloridas_valores == 8)

# Getting the project indices
arr_indicesinicio = np.where(cond)[0] # Returns the index of the array lista_linhas_coloridas_valores where the condition is met
print("Start indices:", arr_indicesinicio)

# Getting the rows of the welded sets
arr_linhasinicio = lista_linhas_coloridas_indices[arr_indicesinicio]
print("Start rows:", arr_linhasinicio)
print()

# I'm just filtering the color following the welded sets and getting the row number

# Slicing the DataFrame using the row array and calculating the costs
for i in range(arr_linhasinicio.shape[0]):
    linhainicio = arr_linhasinicio[i] + 1
    
    if i != arr_linhasinicio.shape[0]-1:
        linhafim = arr_linhasinicio[i+1] 
    else:
        linhafim = max(df.index)

    # Calculates the sum of values within the range
    soma = df.loc[linhainicio:linhafim, "Custo Total"].sum()

    # Updates the value in the specified row
    df.loc[linhainicio - 1, "Custo Total"] = soma

df.to_excel(r"C:\Users\Guilherme Contar\OneDrive - HTMS Group\Documentos\Trabalho python\Validacao.xlsx", index=False)


Indices de início: [ 0  2 16 21]
Linhas de início: [  0   7 141 176]



In [13]:
# Calculating assembled sets

# Getting the index of welded sets and the next color

print("lista_linhas_coloridas_valores", lista_linhas_coloridas_valores)

cond = (lista_linhas_coloridas_valores == 9)

arr_indicesinicio = np.where(cond)[0] # Returns the index of the array lista_linhas_coloridas_valores where theme = 9
print("arr_indicesinicio", arr_indicesinicio)

arr_linhasinicio = lista_linhas_coloridas_indices[arr_indicesinicio] # Returns the row number
print("arr_linhasinicio", arr_linhasinicio)

cond2 = (lista_linhas_coloridas_valores == 9) | (lista_linhas_coloridas_valores == 8)
print("cond2:", lista_linhas_coloridas_valores[cond2])

arr_indicesfim = np.where(cond2)[0]
print("arr_indicesfim:", arr_indicesfim)

arr_linhasfim = lista_linhas_coloridas_indices[arr_indicesfim]
print("arr_linhasfim:", arr_linhasfim)

# Slicing the DataFrame using the row array and calculating the costs
for i in range(1):

    linhainicio = arr_linhasinicio[i] + 1

    for j in range(arr_linhasfim.shape[0]):

        if linhainicio < arr_linhasfim[j]:
            linhafim = arr_linhasfim[j] - 1
            print(linhafim)
            break
            
    # Calculates the sum of values within the range
    print(df.loc[linhainicio:linhafim])
    soma = df.loc[linhainicio:linhafim, "Custo Total"].sum()

    # Updates the value in the specified row
    df.loc[linhainicio - 1, "Custo Total"] = soma


lista_linhas_coloridas_valores [8 2 8 2 9 2 2 9 2 9 9 9 9 9 9 9 8 2 2 9 2 8 2 9 2 9 2 9]
arr_indicesinicio [ 4  7  9 10 11 12 13 14 15 19 23 25 27]
arr_linhasinicio [ 47  61  69  79  86  93 107 119 129 170 201 207 215]
cond2: [8 8 9 9 9 9 9 9 9 9 9 8 9 8 9 9 9]
arr_indicesfim: [ 0  2  4  7  9 10 11 12 13 14 15 16 19 21 23 25 27]
arr_linhasfim: [  0   7  47  61  69  79  86  93 107 119 129 141 170 176 201 207 215]
60
                    CÓDIGO          COMPONENTE                    DESCRIÇÃO  \
48  PA.AL.CL-P12.7X38.1-13                                         FIXAÇÕES   
49      PA.ANT.P12.7X40-13                      PARAF. SEGURANÇA 1/2"X40-13   
50        PA.SX.M8X45-1.25                                         FIXAÇÕES   
51        PO.SX.TAN.M8-1.0                                         FIXAÇÕES   
52      P003.0005.P151_E00                                     ARRUELA 25X8   
53      P003.0005.S130_E00                                  PORTA MÓDULO  1   
54                          

In [14]:
# Calculating welded sets

cond = (lista_linhas_coloridas_valores == 2)

# Getting the index of welded sets
arr_indicesinicio = np.where(cond)[0] # Returns the index of the array lista_linhas_coloridas_valores where the condition is met
print("Start indices:", arr_indicesinicio)

# Getting the rows of the welded sets
arr_linhasinicio = lista_linhas_coloridas_indices[arr_indicesinicio]
print("Start rows:", arr_linhasinicio)
print()

# I'm just filtering what is a welded set and getting the row number of each

# Getting the index following the welded set
arr_indicefim = arr_indicesinicio + 1 # Gets the index of the next colored row
print("End indices:", arr_indicefim)

# Getting the rows of the next color
arr_linhasfim = lista_linhas_coloridas_indices[arr_indicefim]
print("End rows:", arr_linhasfim)

# I'm just filtering the color following the welded sets and getting the row number

# Slicing the DataFrame using the row array and calculating the costs
for i in range(arr_linhasinicio.shape[0]):
    linhainicio = arr_linhasinicio[i] + 1
    linhafim = arr_linhasfim[i]

    # Calculates the sum of values within the range
    soma = df.loc[linhainicio:linhafim, "Custo Total"].sum()

    # Updates the value in the specified row
    df.loc[linhainicio - 1, "Custo Total"] = soma


Indices de início: [ 1  3  5  6  8 17 18 20 22 24 26]
Linhas de início: [  3  16  53  58  66 151 165 172 185 203 212]

Indices de fim: [ 2  4  6  7  9 18 19 21 23 25 27]
Linhas de fim: [  7  47  58  61  69 165 170 176 201 207 215]
